In [ ]:
git clone https://github.com/facebookresearch/detectron2.git

In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Cloning into 'detectron2''...
fatal: protocol ''https' is not supported
The system cannot find the file specified.


In [2]:

# PyTorch 설치 (CUDA 11.8 기준, 필요 시 공식 사이트에서 버전 확인)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Detectron2 설치 (Colab 환경)
!pip install torch torchvision
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

# OpenCV 및 기타 라이브러리 설치
!pip install opencv-python numpy json5 tqdm matplotlib datasets

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
pip install fvcore 
pip install cloudpickle
pip install omegaconf 
pip install pycocotools

In [8]:
# 필요한 라이브러리 불러오기
import os
import json
import cv2
import numpy as np
import torch
import random
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode

In [11]:
import os
import json
from tqdm import tqdm

# 원본 JSON 폴더 및 이미지 폴더 경로
json_dir = r"C:\Users\nature\Documents\detectron2\dp-bench\dataset\json"
image_dir = r"C:\Users\nature\Documents\detectron2\dp-bench\dataset\images"

# COCO JSON 저장 경로
coco_output_path = r"C:\Users\nature\Documents\detectron2\dp-bench\coco\coco_annotations.json"

# COCO JSON 구조 초기화
coco_format = {
    "images": [],
    "annotations": [],
    "categories": []
}

# 카테고리 매핑
category_mapping = {}
category_id = 1
annotation_id = 1
image_id = 1

# JSON 파일 목록 가져오기
json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for json_file in tqdm(json_files, desc="Processing JSON Files"):
    json_path = os.path.join(json_dir, json_file)
    
    with open(json_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)

    for pdf_name, pdf_data in dataset.items():
        for element in pdf_data["elements"]:
            page_num = element["page"]
            img_filename = f"{pdf_name.replace('.pdf', '')}_page_{page_num}.jpg"

            # 이미지 정보 추가
            if not any(img["file_name"] == img_filename for img in coco_format["images"]):
                coco_format["images"].append({
                    "id": image_id,
                    "file_name": img_filename,
                    "width": 2480,
                    "height": 3508
                })

            # 카테고리 추가
            label = element["category"]
            if label not in category_mapping:
                category_mapping[label] = category_id
                coco_format["categories"].append({"id": category_id, "name": label})
                category_id += 1

            # 좌표 변환 (Bounding Box 변환)
            x_min = min(coord["x"] for coord in element["coordinates"]) * 2480
            y_min = min(coord["y"] for coord in element["coordinates"]) * 3508
            x_max = max(coord["x"] for coord in element["coordinates"]) * 2480
            y_max = max(coord["y"] for coord in element["coordinates"]) * 3508

            bbox = [x_min, y_min, x_max - x_min, y_max - y_min]

            # ✅ `segmentation` 필드 추가 (간단한 다각형 형태로 변환)
            segmentation = [[
                x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max
            ]]

            # 어노테이션 추가
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_mapping[label],
                "bbox": bbox,
                "area": bbox[2] * bbox[3],
                "segmentation": segmentation,  # ✅ 추가됨
                "iscrowd": 0,
                "text": element["content"]["text"]
            })
            annotation_id += 1

        image_id += 1

# COCO JSON 저장
with open(coco_output_path, "w", encoding="utf-8") as f:
    json.dump(coco_format, f, indent=4)

print(f"✅ COCO 데이터 저장 완료: {coco_output_path}")


Processing JSON Files: 100%|██████████| 1/1 [00:25<00:00, 25.24s/it]

✅ COCO 데이터 저장 완료: C:\Users\nature\Documents\detectron2\dp-bench\coco\coco_annotations.json


In [ ]:
import os
from pdf2image import convert_from_path
from tqdm import tqdm

# PDF 및 이미지 디렉토리 설정
pdf_dir = r"C:\Users\nature\Documents\detectron2\dp-bench\dataset\pdfs"
image_output_dir = r"C:\Users\nature\Documents\detectron2\dp-bench\dataset\images"

os.makedirs(image_output_dir, exist_ok=True)

# PDF 파일 목록 가져오기
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

# PDF를 이미지로 변환
for pdf_file in tqdm(pdf_files, desc="Converting PDFs to Images"):
    pdf_path = os.path.join(pdf_dir, pdf_file)
    images = convert_from_path(pdf_path, dpi=300)  # PDF를 고해상도 이미지로 변환

    # 변환된 이미지를 저장
    for i, img in enumerate(images):
        img_filename = f"{pdf_file.replace('.pdf', '')}_page_{i+1}.jpg"
        img_path = os.path.join(image_output_dir, img_filename)
        img.save(img_path, "JPEG")

print("PDF -> 이미지 변환 완료!")

Converting PDFs to Images: 100%|██████████| 200/200 [01:03<00:00,  3.15it/s]

PDF -> 이미지 변환 완료!


🔹 2. Detectron2에서 Custom COCO Dataset 등록

In [12]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# Detectron2 데이터셋 등록
register_coco_instances("custom_doclaynet", {}, coco_output_path, image_dir)

# 데이터셋 확인
dataset_metadata = MetadataCatalog.get("custom_doclaynet")
dataset_dicts = DatasetCatalog.get("custom_doclaynet")

print("Detectron2 데이터셋 등록 완료!")

[02/12 06:54:18 d2.data.datasets.coco]: Loaded 200 images in COCO format from C:\Users\nature\Documents\detectron2\dp-bench\coco\coco_annotations.json
Detectron2 데이터셋 등록 완료!


In [13]:
from detectron2.data import MetadataCatalog

# COCO 데이터셋 등록 후, thing_classes를 직접 추가
metadata = MetadataCatalog.get("custom_doclaynet")

# COCO JSON에서 카테고리 목록 가져오기
import json

coco_json_path = (
    r"C:\Users\nature\Documents\detectron2\dp-bench\coco\coco_annotations.json"
)
with open(coco_json_path, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# COCO JSON에서 카테고리 추출하여 thing_classes 설정
categories = [cat["name"] for cat in coco_data["categories"]]
metadata.thing_classes = categories  # ✅ 직접 설정

print("카테고리 목록:", metadata.thing_classes)

카테고리 목록: ['table', 'paragraph', 'heading1', 'header', 'list', 'figure', 'caption', 'chart', 'footer', 'footnote', 'equation', 'index']


In [14]:
import os
import cv2
import json

# 이미지 및 COCO JSON 파일 경로
image_dir = r"C:\Users\nature\Documents\detectron2\dp-bench\dataset\images"
coco_json_path = r"C:\Users\nature\Documents\detectron2\dp-bench\coco\coco_annotations.json"

# COCO JSON 파일 로드
with open(coco_json_path, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# 이미지 크기 확인 및 COCO JSON 수정
for img in coco_data["images"]:
    img_path = os.path.join(image_dir, img["file_name"])

    if os.path.exists(img_path):
        # 이미지 로드 및 크기 확인
        image = cv2.imread(img_path)
        height, width, _ = image.shape

        # COCO JSON 업데이트
        img["width"] = width
        img["height"] = height

# 수정된 COCO JSON 저장
with open(coco_json_path, "w", encoding="utf-8") as f:
    json.dump(coco_data, f, indent=4)

print("✅ COCO JSON 업데이트 완료! 이제 Detectron2에서 실행해보세요.")


✅ COCO JSON 업데이트 완료! 이제 Detectron2에서 실행해보세요.


In [16]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Mask R-CNN 학습 설정
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
)
cfg.DATASETS.TRAIN = ("custom_doclaynet",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)  # 사전학습 모델 사용
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 10  # 학습 반복 횟수
cfg.MODEL.MASK_ON = True
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(dataset_metadata.thing_classes)

# 학습 실행
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[02/12 06:55:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (13, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (13,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (48, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (48,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (12, 256, 1, 

[02/12 06:55:50 d2.engine.train_loop]: Starting training from iteration 0
[02/12 06:55:58 d2.utils.events]:  eta: 0:00:00  iter: 9  total_loss: 7.325  loss_cls: 2.508  loss_box_reg: 0.1683  loss_mask: 0.6894  loss_rpn_cls: 3.279  loss_rpn_loc: 0.588    time: 0.3318  last_time: 0.3730  data_time: 0.4346  last_data_time: 0.0014   lr: 0.00022503  max_mem: 4671M
[02/12 06:55:58 d2.engine.hooks]: Overall training speed: 8 iterations in 0:00:02 (0.3318 s / it)
[02/12 06:55:58 d2.engine.hooks]: Total training time: 0:00:03 (0:00:00 on hooks)


In [17]:
import os
import shutil

# 모델 저장 경로 설정
output_dir = cfg.OUTPUT_DIR
model_final_path = os.path.join(output_dir, "model_final.pth")  # 최종 모델 파일

# 모델이 존재하는지 확인 후 백업
if os.path.exists(model_final_path):
    save_path = r"C:\Users\nature\Documents\detectron2\trained_model\mask_rcnn_model.pth"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    shutil.copy(model_final_path, save_path)
    print(f"✅ 모델이 저장되었습니다: {save_path}")
else:
    print("⚠️ 모델 파일을 찾을 수 없습니다. 학습이 완료되었는지 확인하세요.")


✅ 모델이 저장되었습니다: C:\Users\nature\Documents\detectron2\trained_model\mask_rcnn_model.pth


In [1]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    print("✅ GPU 사용 가능:", torch.cuda.get_device_name(0))
else:
    print("⚠️ GPU가 비활성화됨! CPU에서 실행 중.")

# Detectron2 설정에서 GPU 사용하도록 강제 설정
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


✅ GPU 사용 가능: NVIDIA GeForce RTX 2070 SUPER


NameError: name 'cfg' is not defined

In [ ]:
import cv2
import torch
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
save_path =r"C:\Users\nature\Documents\detectron2\trained_model\mask_rcnn_model.pth"
# 모델 로드
# Mask R-CNN 학습 설정
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
)
cfg.MODEL.WEIGHTS = save_path  # 학습된 모델 가중치 불러오기
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # 신뢰도 임계값 설정
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # GPU 사용 가능 여부 확인

# 예측을 위한 Predictor 생성
predictor = DefaultPredictor(cfg)

# 테스트할 이미지 불러오기
test_image_path = r"C:\Users\nature\Documents\detectron2\dp-bench\dataset\images\01030000000037_page_1.jpg"
image = cv2.imread(test_image_path)

# 예측 실행
outputs = predictor(image)

# 결과 시각화
v = Visualizer(image[:, :, ::-1], metadata=MetadataCatalog.get("custom_doclaynet"), scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# 예측된 이미지 출력
cv2.imshow("Predictions", v.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()


[02/12 07:03:18 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\nature\Documents\detectron2\trained_model\mask_rcnn_model.pth ...


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (13, 1024) in the checkpoint but (81, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (13,) in the checkpoint but (81,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (48, 1024) in the checkpoint but (320, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (48,) in the checkpoint but (320,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (12, 256, 1, 1) in the checkpoint but (80, 256, 1, 

 3. 학습된 모델을 나중에 다시 로드하는 방법
이제 저장된 모델을 나중에 다시 로드하여 추가 훈련하거나 다른 데이터에 대해 예측할 수도 있어.

In [5]:
from detectron2.engine import DefaultPredictor

# 기존 모델 불러오기
cfg.MODEL.WEIGHTS = r"C:\Users\nature\Documents\detectron2\trained_model\mask_rcnn_model.pth"
predictor = DefaultPredictor(cfg)

# 새로운 이미지 예측
new_image_path = r"C:\Users\nature\Documents\detectron2\doclaynet\images\01030000000045_page_2.jpg"
image = cv2.imread(new_image_path)
outputs = predictor(image)

# 결과 확인
v = Visualizer(image[:, :, ::-1], metadata=MetadataCatalog.get("custom_doclaynet"), scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2.imshow("Predictions", v.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'cfg' is not defined